# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.7392394480231195
 1.152219127766849
 0.8282779345485132

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
```
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.158625  1.0       0.0
 0.154961  0.958186  1.0
U factor:
3×3 Matrix{Float64}:
 0.494193  0.610149   0.634898
 0.0       0.57543    0.300903
 0.0       0.0       -0.280925

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.158625  1.0       0.0
 0.154961  0.958186  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.494193  0.610149   0.634898
 0.0       0.57543    0.300903
 0.0       0.0       -0.280925

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 1.0
 1.0000000000000004
 0.9999999999999994

In [8]:
Alu\b

3-element Vector{Float64}:
 1.0
 1.0000000000000004
 0.9999999999999994

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
```

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`.

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.976284   0.216484  -0.00214357
 -0.154863  -0.70524   -0.691848
 -0.151286  -0.675108   0.72204
R factor:
3×3 Matrix{Float64}:
 -0.506198  -0.797498  -0.698038
  0.0       -0.77805   -0.217201
  0.0        0.0       -0.202839

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.976284   0.216484  -0.00214357
 -0.154863  -0.70524   -0.691848
 -0.151286  -0.675108   0.72204

In [12]:
Aqr.R

3×3 Matrix{Float64}:
 -0.506198  -0.797498  -0.698038
  0.0       -0.77805   -0.217201
  0.0        0.0       -0.202839

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.47428666136326614
  0.4597423842385169
  2.5589183450775987
vectors:
3×3 Matrix{Float64}:
 -0.230128   0.819539   -0.524783
 -0.418825  -0.570167   -0.70675
  0.878423  -0.0571497  -0.474454

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -0.47428666136326614
  0.4597423842385169
  2.5589183450775987

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.230128   0.819539   -0.524783
 -0.418825  -0.570167   -0.70675
  0.878423  -0.0571497  -0.474454

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [18]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0           9.4369e-16  -1.26288e-15
  1.77636e-15   1.0         -2.47025e-15
 -1.55431e-15  -1.9984e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [19]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [20]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [21]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.27790400214257077

In [22]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [23]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [24]:
@time eigvals(Asym);

  0.221902 seconds (134.03 k allocations: 15.456 MiB, 9.02% gc time, 77.00% compilation time)


In [25]:
@time eigvals(Asym_noisy);

  0.541851 seconds (13 allocations: 7.920 MiB)


In [26]:
@time eigvals(Asym_explicit);

  0.074539 seconds (5.88 k allocations: 8.358 MiB, 8.36% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [27]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.825493 seconds (823.17 k allocations: 229.291 MiB, 22.39% gc time, 25.50% compilation time)


6.458362819153823

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [ ]:
1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [ ]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

In [ ]:
x = fill(1, 3)
b = Arational*x

In [ ]:
Arational\b

In [ ]:
lu(Arational)

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [ ]:
using LinearAlgebra

In [ ]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [ ]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]
# #### 11.3
# Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]